# Stencil Python Serial

In [1]:
import sys
print (sys.version)
print (sys.version_info)

3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 21:52:21) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [2]:
import numpy
numpy.show_config()

blas_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
blas_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_mkl_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']
lapack_opt_info:
    libraries = ['mkl_rt', 'pthread']
    library_dirs = ['/scratch/app/anaconda3/2018.12/lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['/scratch/app/anaconda3/2018.12/include']


#### Versão sequencial

In [1]:
%%time
from time import time
t = time()
import numpy as np

n            = 4800    # nxn grid (4800,1,500)=1500
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

size         = n + 2
sizeEnd      = n + 1
anew = aold  = np.zeros((size,  size), np.float64)
nsources     = 3       # sources of energy
sources      = np.empty((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters+1) // 2

for iters in range(niters):
    anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                        ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                          aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
    anew[sources[0:nsources,0], sources[0:nsources,1]] += energy     
    aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 +
                        ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                          anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
    aold[sources[0:nsources,0], sources[0:nsources,1]] += energy   
heat = np.sum( aold[1:sizeEnd, 1:sizeEnd] )  # system total heat

t = time() - t
print("Heat = %0.4f | Tempo = %0.4f s" %(heat, t))

Heat = 1500.0000 | Tempo = 237.5668 s
CPU times: user 2min 27s, sys: 1min 29s, total: 3min 57s
Wall time: 4min 2s


### Gravando em um arquivo no disco

In [5]:
%%writefile stc_pyt_seq.py
from time import time
t = time()
import numpy as np

n            = 4800    # nxn grid (4800,1,500)=1500
energy       = 1.0     # energy to be injected per iteration
niters       = 500     # number of iterations

size         = n + 2
sizeEnd      = n + 1
anew = aold  = np.zeros((size,  size), np.float64)
nsources     = 3       # sources of energy
sources      = np.empty((nsources, 2), np.int)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
niters       = (niters+1) // 2

for iters in range(niters):
    anew[1:-1, 1:-1] =  ( aold[1:-1, 1:-1] / 2.0 + 
                        ( aold[2:  , 1:-1] + aold[ :-2, 1:-1] +
                          aold[1:-1, 2:  ] + aold[1:-1,  :-2] ) / 8.0 )
    anew[sources[0:nsources,0], sources[0:nsources,1]] += energy     
    aold[1:-1, 1:-1] =  ( anew[1:-1, 1:-1] / 2.0 +
                        ( anew[2:  , 1:-1] + anew[ :-2, 1:-1] +
                          anew[1:-1, 2:  ] + anew[1:-1,  :-2] ) / 8.0 )
    aold[sources[0:nsources,0], sources[0:nsources,1]] += energy   
heat = np.sum( aold[1:sizeEnd, 1:sizeEnd] )  # system total heat

t = time() - t
print("Heat = %0.4f | Tempo = %0.4f s" %(heat, t))

Overwriting stc_pyt_seq.py


In [1]:
%%writefile calctempo.sh
#!/bin/sh
time python stc_pyt_seq.py

Writing calctempo.sh


### Testando o funcionamento

In [2]:
%%bash
sh calctempo.sh

Heat = 1500.0000 | Tempo = 220.3673 s



real	3m45.647s
user	2m21.746s
sys	1m18.986s


### Copia para scratch

In [3]:
%%bash
s='/prj/ampemi/eduardo.miranda2/stnc/Python'
d='/scratch/ampemi/eduardo.miranda2/stnc/Python'
cp  $s/stc_pyt_seq.py $s/calctempo.sh $d

### Arquivo de submissão

In [4]:
%%writefile stc_pyt_seq.srm
#!/bin/bash
# limites das filas (1,0 UA):
#   cpu_dev  : 20 min.,  1-4  nós, 1/1   tarefas em exec/fila máximo
#   cpu_small: 72 horas, 1-20 nós, 16/96 tarefas em exec/fila máximo
#SBATCH -p cpu_small           #Fila (partition) a ser utilizada
#SBATCH --ntasks=1             #Total de tarefas
#SBATCH --nodes=1              #Qtd de nós
#SBATCH --ntasks-per-node=1    #Qtd de tarefas por nó ($SLURM_NTASKS_PER_NODE)
#SBATCH -J stpyseq             #Nome do job, 8 caracteres
# #SBATCH --exclusive            #Utilização exclusiva dos nós
#SBATCH --time=00:05:00        #Tempo max. de execução

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
echo '- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):'
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR

#Entra no diretório de trabalho
cd /scratch/ampemi/eduardo.miranda2/stnc/Python

#Executavel
EXEC='sh calctempo.sh'

#Dispara a execucao
echo '-- srun -------------------------------'
echo '$ srun -n ' $SLURM_NTASKS $EXEC
srun -n $SLURM_NTASKS $EXEC
echo '-- FIM --------------------------------'

Overwriting stc_pyt_seq.srm


## Envia para execução

In [5]:
%%bash
sbatch stc_pyt_seq.srm
sbatch stc_pyt_seq.srm
sbatch stc_pyt_seq.srm
squeue -n stpyseq

Submitted batch job 772572
Submitted batch job 772573
Submitted batch job 772574
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772572 cpu_small  stpyseq eduardo. PD       0:00      1 (Priority)
            772573 cpu_small  stpyseq eduardo. PD       0:00      1 (Priority)
            772574 cpu_small  stpyseq eduardo. PD       0:00      1 (Priority)


### Verifica se já terminou

In [7]:
%%bash
squeue -n stpyseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


### Mostra os arquivos de saída

In [8]:
%%bash
b='/stnc/Python'
d='/scratch/ampemi/eduardo.miranda2/'$b
cat $d/slurm-772572.out
cat $d/slurm-772573.out
cat $d/slurm-772574.out

- Job ID: 772572
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1405
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 225.1292 s

real	3m46.153s
user	2m56.529s
sys	0m48.873s
-- FIM --------------------------------
- Job ID: 772573
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1306
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun -------------------------------
$ srun -n  1 sh calctempo.sh
Heat = 1500.0000 | Tempo = 187.1582 s

real	3m7.998s
user	2m24.770s
sys	0m42.647s
-- FIM --------------------------------
- Job ID: 772574
- Tarefas por no: 1
- Qtd. de nos: 1
- Tot. de tarefas: 1
- Nos alocados: sdumont1405
- diretorio onde sbatch foi chamado ($SLURM_SUBMIT_DIR):
/prj/ampemi/eduardo.miranda2/stnc/Python
-- srun ------

In [1]:
%%bash
sbatch stc_pyt_seq.srm
sbatch stc_pyt_seq.srm
squeue -n stpyseq

Submitted batch job 772188
Submitted batch job 772189
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772188 cpu_small  stpyseq eduardo. PD       0:00      1 (Priority)
            772189 cpu_small  stpyseq eduardo. PD       0:00      1 (Priority)


In [2]:
%%bash
squeue -n stpyseq

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            772188 cpu_small  stpyseq eduardo. PD       0:00      1 (Resources)
            772189 cpu_small  stpyseq eduardo. PD       0:00      1 (Resources)
